In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

In [2]:
# ------------------------
# Step 1: Data Preparation
# ------------------------
data_dir = "database"  # folder with dysgraphia/ and normal/
batch_size = 16
img_size = 224

transform = {
    "train": transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

train_dataset = datasets.ImageFolder(root=data_dir, transform=transform["train"])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [3]:
# ------------------------
# Step 2: Define Model
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

# Replace last FC layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\Server\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Server\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Server/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [4]:
# ------------------------
# Step 3: Training Loop
# ------------------------
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss, correct = 0.0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

# Save model
torch.save(model.state_dict(), "dysgraphia_model.pth")

Epoch 1/5, Loss: 0.8154, Acc: 0.7510
Epoch 2/5, Loss: 0.3045, Acc: 0.8554
Epoch 3/5, Loss: 0.2115, Acc: 0.9237
Epoch 4/5, Loss: 0.1766, Acc: 0.9438
Epoch 5/5, Loss: 0.1668, Acc: 0.9357


In [14]:
# ------------------------
# Step 4: Prediction
# ------------------------
def predict_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)
    
    return class_names[pred.item()]

# Example usage
class_names = train_dataset.classes  # ["dysgraphia", "normal"]
test_img = r"C:\Users\Server\Documents\dysgraphia-detector\ml\Camera_ai\snips\snip_20250927_170314_610704.png"  # your image
prediction = predict_image(test_img, model, transform["val"], class_names)
print("Prediction:", prediction)

Prediction: Dysgraphia


In [11]:
torch.save(model.state_dict(), "old_dysgraphia_model.pth")